In [2]:
import requests
import json
import time
from tqdm import tqdm
import pandas as pd
import random

In [2]:
url = 'https://api.hh.ru/areas/113' # Россия
res = requests.get(url)
areas = res.json()

In [3]:
areas

{'id': '113',
 'parent_id': None,
 'name': 'Россия',
 'areas': [{'id': '1620',
   'parent_id': '113',
   'name': 'Республика Марий Эл',
   'areas': [{'id': '4228',
     'parent_id': '1620',
     'name': 'Виловатово',
     'areas': []},
    {'id': '1621', 'parent_id': '1620', 'name': 'Волжск', 'areas': []},
    {'id': '1622', 'parent_id': '1620', 'name': 'Звенигово', 'areas': []},
    {'id': '4229', 'parent_id': '1620', 'name': 'Знаменский', 'areas': []},
    {'id': '61', 'parent_id': '1620', 'name': 'Йошкар-Ола', 'areas': []},
    {'id': '4230', 'parent_id': '1620', 'name': 'Кельмаксола', 'areas': []},
    {'id': '4231', 'parent_id': '1620', 'name': 'Килемары', 'areas': []},
    {'id': '1623', 'parent_id': '1620', 'name': 'Козьмодемьянск', 'areas': []},
    {'id': '4232',
     'parent_id': '1620',
     'name': 'Красногорский (Республика Марий Эл)',
     'areas': []},
    {'id': '4233',
     'parent_id': '1620',
     'name': 'Краснооктябрьский',
     'areas': []},
    {'id': '4234', 'pa

In [4]:
parent_ids = [el.get('id') for el in areas.get('areas')]

In [5]:
df=pd.DataFrame()
url = 'https://api.hh.ru/areas/{}'
area = []
for i in (parent_ids):
    df=df.append(pd.json_normalize(requests.get(url.format(i)).json()))
df.to_excel("area.xlsx")

In [6]:
area_ids = []
for i in df.iloc():
    for j in i['areas']:
        area_ids.append(j['id'])

In [10]:
len(area_ids)

3925

In [11]:
%%time
vacancy_ids = []
# страница
page = 0
# кол-во записей на странице
num_per_page = 100
# ссылка
for i in range(0, 20):
    for j in random.choices(area_ids, k = 1000):
        url = f'https://api.hh.ru/vacancies?page={i}&per_page={num_per_page}&area={j}'
        res = requests.get(url)
        vacancies = res.json()
        num_pages = vacancies.get('pages')
        vacancy_id = [el.get('id') for el in vacancies.get('items')]
        vacancy_ids.extend(vacancy_id)
        time.sleep(0.5)

TypeError: 'NoneType' object is not iterable

In [16]:
len(vacancy_ids)

51566

In [17]:
vacancy_ids_df=pd.DataFrame(vacancy_ids)

In [13]:
vacancy_ids_df.to_excel("vacancy_ids.xlsx")

In [19]:
vacancies_df=pd.DataFrame()
url = 'https://api.hh.ru/vacancies/{}'
for i in tqdm(random.choices(vacancy_ids, k = 10000)):
    vacancies_df=vacancies_df.append(pd.json_normalize(requests.get(url.format(i)).json()))
    time.sleep(0.0001)
vacancies_df.to_csv("vacancies_10000.csv")

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [1:04:23<00:00,  2.59it/s]


In [3]:
vacancies_df = pd.read_csv('vacancies_10000.csv')

C:\Users\comp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,3,4,6,7,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,28,29,31,32,33,34,35,36,37,39,40,43,44,45,46,47,48,49,53,55,56,57,58,59,60,61,62,63,64,65,67,68,69,70,71,72,73,74,79,80,81,86,88,91,92,95,96,98) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
vacancies_df.head()

Unnamed: 0          id premium relations  \
0           0  51106445.0   False        []   
1           0  51306804.0   False        []   
2           0  48303287.0   False        []   
3           0  49504814.0   False        []   
4           0  50680737.0   False        []   

                                                name  insider_interview  \
0      Директор магазина (Ялуторовск, Революции, 65)                NaN   
1                Мерчендайзер/Продавец - консультант                NaN   
2               Курьер-водитель с личным автомобилем                NaN   
3                         Продавец-кассир в ТЦ Город                NaN   
4  Монтажно-сварочная бригада (технологическое об...                NaN   

  response_letter_required allow_messages  contacts  \
0                    False          False       NaN   
1                    False           True       NaN   
2                    False           True       NaN   
3                    False           True       NaN   
4                    False           True       NaN   

                                         description  ...  \
0  <p><strong>Для каждого, кто хочет работать и з...  ...   
1  <p><strong>Обязанности</strong>:</p> <ul> <li>...  ...   
2  <p><strong>ПРИГЛАШАЕМ НА РАБОТУ КУРЬЕРОВ С ЛИЧ...  ...   
3  <em><strong>Обязанности:</strong></em> <ul> <l...  ...   
4  <strong>Обязанности:</strong> <ul> <li>Работа ...  ...   

  vacancy_constructor_template.top_picture.path  \
0                                           NaN   
1                                           NaN   
2                                           NaN   
3                                           NaN   
4                                           NaN   

   vacancy_constructor_template.top_picture.blurred_path  \
0                                                NaN       
1                                                NaN       
2                                                NaN       
3                                                NaN       
4                                                NaN       

  vacancy_constructor_template.bottom_picture.height  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   

  vacancy_constructor_template.bottom_picture.width  \
0                                               NaN   
1                                               NaN   
2                                               NaN   
3                                               NaN   
4                                               NaN   

  vacancy_constructor_template.bottom_picture.path  \
0                                              NaN   
1                                              NaN   
2                                              NaN   
3                                              NaN   
4                                              NaN   

  vacancy_constructor_template.bottom_picture.blurred_path  \
0                                                NaN         
1                                                NaN         
2                                                NaN         
3                                                NaN         
4                                                NaN         

  insider_interview.id insider_interview.url errors request_id  
0                  NaN                   NaN    NaN        NaN  
1                  NaN                   NaN    NaN        NaN  
2                  NaN                   NaN    NaN        NaN  
3                  NaN                   NaN    NaN        NaN  
4                  NaN                   NaN    NaN        NaN  

[5 rows x 101 columns]

In [9]:
vacancies_df['description'].isna().sum()

6000

In [10]:
vac_df = vacancies_df[~vacancies_df.description.isna()]

In [17]:
vac_df['description']

0       <p><strong>Для каждого, кто хочет работать и з...
1       <p><strong>Обязанности<strong>:<p> <ul> <li>вы...
2       <p><strong>ПРИГЛАШАЕМ НА РАБОТУ КУРЬЕРОВ С ЛИЧ...
3       <em><strong>Обязанности:<strong><em> <ul> <li>...
4       <strong>Обязанности:<strong> <ul> <li>Работа н...
                              ...                        
3995    <strong>Обязанности:<strong> <ul> <li>Формиров...
3996    <p>Для работы в сети стоматологических клиник ...
3997    <p><strong>Крупная юридическая компания в связ...
3998    <p>Требуется менеджер по продажам в магазин ав...
3999    <strong>Требования:<strong> <ul> <li>умение ра...
Name: description, Length: 4000, dtype: object

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn import tree

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\comp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [24]:
stop_russian = stopwords.words('russian')

In [25]:
description_transformer = TfidfVectorizer(stop_words=stop_russian, ngram_range=(1,1), lowercase=True, max_features=10000)

In [26]:
description = description_transformer.fit_transform(vac_df['description'])

In [31]:
description.shape

(4000, 10000)

In [34]:
type(description)

scipy.sparse.csr.csr_matrix

In [ ]:
def remove_trash(list): 
    pattern = r'[^А-Яа-я0-9]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
      print (list)
    except Exception as e:
      print(e)
    return list